In [ ]:
%reload_ext autoreload
%autoreload 2


import functools
print = functools.partial(print, flush=True)

import os
import numpy as np
import pandas as pd
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42 # for pdfs
import matplotlib.pyplot as plt
from matplotlib import cm
from pathlib import Path
import pickle
from tqdm import tqdm
import scipy

import flexiznam as flz
from cottage_analysis.preprocessing import synchronisation
from cottage_analysis.analysis import spheres, find_depth_neurons, common_utils, fit_gaussian_blob, size_control
from cottage_analysis.plotting import basic_vis_plots, plotting_utils
from cottage_analysis.pipelines import pipeline_utils

In [ ]:
def plot_raster_all_depths(
    fig, 
    neurons_df,
    trials_df,
    roi,
    is_closed_loop,
    max_distance=6,
    nbins=60,
    frame_rate=15,
    vmax=1,
    plot=True,
    plot_x=0,
    plot_y=0,
    plot_width=1,
    plot_height=1,
    cbar_width=0.05,
    fontsize_dict={"title": 15, "label": 10, "tick": 10},
):
    """Raster plot for neuronal activity for each depth

    Args:
        fig (plt.figure): figure to plot on.
        neurons_df (pd.DataFrame): dataframe with analyzed info of all rois.
        trials_df (pd.DataFrame): dataframe with info of all trials.
        roi (int): ROI number
        is_closed_loop (bool): plotting the closed loop or open loop results.
        max_distance (int, optional): max distance for each trial in meters. Defaults to 6.
        nbins (int, optional): number of bins to bin the activity. Defaults to 60.
        frame_rate (int, optional): imaging frame rate. Defaults to 15.
        vmax (int, optional): vmax to plot the heatmap. Defaults to 1.
        plot (bool, optional): whether to plot or not. Defaults to True.
        plot_x (int, optional): x position of the plot out of 1. Defaults to 0.
        plot_y (int, optional): y position of the plot out of 1. Defaults to 0.
        plot_width (int, optional): width of the plot out of 1. Defaults to 1.
        plot_height (int, optional): height of the plot out of 1. Defaults to 1.
        cbar_width (float, optional): width of the colorbar. Defaults to 0.05.
        fontsize_dict (dict, optional): dictionary of fontsize for title, label and tick. Defaults to {"title": 20, "label": 15, "tick": 15}.
    """
    # choose the trials with closed or open loop to visualize
    trials_df = trials_df[trials_df.closed_loop == is_closed_loop]

    depth_list = find_depth_neurons.find_depth_list(trials_df)
    grouped_trials = trials_df.groupby(by="depth")
    trial_number = len(trials_df) // len(depth_list)

    # bin dff according to distance travelled for each trial
    dffs_binned = np.zeros((len(depth_list), trial_number, nbins))
    for idepth, depth in enumerate(depth_list):
        all_dffs = grouped_trials.get_group(depth).dff_stim.values
        all_rs = grouped_trials.get_group(depth).RS_stim.values
        for itrial in np.arange(len(all_dffs)):
            if itrial < trial_number:
                dff = all_dffs[itrial][:, roi]
                rs_arr = all_rs[itrial]
                distance = np.cumsum(rs_arr / frame_rate)
                bins = np.linspace(
                    start=0, stop=max_distance, num=nbins + 1, endpoint=True
                )
                bin_means, _, _ = scipy.stats.binned_statistic(
                    x=distance,
                    values=dff,
                    statistic="mean",
                    bins=nbins,
                )
                dffs_binned[idepth, itrial, :] = bin_means

    # colormap
    WhRdcmap = basic_vis_plots.generate_cmap(cmap_name="WhRd")

    # plot each depth as a heatmap
    if plot:
        each_plot_width = (plot_width-cbar_width)/ len(depth_list)
        for idepth, depth in enumerate(depth_list):
            ax = fig.add_axes([plot_x+idepth*each_plot_width, plot_y, each_plot_width*0.8, plot_height]) 
            im = ax.imshow(
                dffs_binned[idepth], aspect="auto", cmap=WhRdcmap, vmin=0, vmax=vmax
            )
            plt.xticks(
                np.linspace(0, nbins, 3),
                (np.linspace(0, max_distance, 3) * 100).astype("int"),
            )
            if idepth == 0:
                ax.set_ylabel("Trial no.", fontsize=fontsize_dict["label"])
                ax.tick_params(
                    left=True,
                    right=False,
                    labelleft=True,
                    labelbottom=True,
                    bottom=True,
                    labelsize=fontsize_dict["tick"],
                )
                ax.tick_params(axis="x", rotation=45)
            else:
                ax.tick_params(
                    left=True,
                    right=False,
                    labelleft=False,
                    labelbottom=True,
                    bottom=True,
                    labelsize=fontsize_dict["tick"],
                )
                ax.tick_params(axis="x", rotation=45)
            if idepth == len(depth_list)//2:
                ax.set_xlabel("Virtual distance (cm)", fontsize=fontsize_dict["label"])

        ax2 = fig.add_axes([plot_x+len(depth_list)*each_plot_width, plot_y, cbar_width*0.8, plot_height]) 
        fig.colorbar(im, cax=ax2, label="\u0394F/F")
                
    return dffs_binned


def plot_depth_tuning_curve(
    fig,
    neurons_df,
    trials_df,
    roi,
    rs_thr=0.2,
    plot_fit=True,
    linewidth=3,
    linecolor="k",
    fit_linecolor="r",
    closed_loop=1,
    plot_x=0,
    plot_y=0,
    plot_width=1,
    plot_height=1,
    fontsize_dict={"title": 15, "label": 10, "tick": 10},
):
    """
    Plot depth tuning curve for one neuron.

    Args:
        neurons_df (pd.DataFrame): dataframe with analyzed info of all rois.
        trials_df (pd.DataFrame): dataframe with info of all trials.
        roi (int): ROI number
        rs_thr (float, optional): Threshold to cut off non-running frames. Defaults to 0.2. (m/s)
        plot_fit (bool, optional): Whether to plot fitted tuning curve or not. Defaults to True.
        linewidth (int, optional): linewidth. Defaults to 3.
        linecolor (str, optional): linecolor of true data. Defaults to "k".
        fit_linecolor (str, optional): linecolor of fitted curve. Defaults to "r".
    """

    # Load average activity and confidence interval for this roi
    depth_list = np.array(find_depth_neurons.find_depth_list(trials_df))
    mean_dff_arr = find_depth_neurons.average_dff_for_all_trials(
        trials_df, rs_thr=rs_thr
    )[:, :, roi]
    CI_low, CI_high = common_utils.get_confidence_interval(mean_dff_arr)
    mean_arr = np.mean(mean_dff_arr, axis=1)

    # Load gaussian fit params for this roi
    if plot_fit:
        min_sigma = 0.5
        [a, x0, log_sigma, b] = neurons_df.loc[roi, "depth_tuning_popt_closedloop"]
        x = np.geomspace(depth_list[0], depth_list[-1], num=100)
        gaussian_arr = find_depth_neurons.gaussian_func(
            np.log(x), a, x0, log_sigma, b, min_sigma
        )

    # Plotting
    ax = fig.add_axes([plot_x, plot_y, plot_width, plot_height]) 
    ax.plot(np.log(depth_list), mean_arr, color=linecolor, linewidth=linewidth)
    ax.fill_between(
        np.log(depth_list),
        CI_low,
        CI_high,
        color=linecolor,
        alpha=0.3,
        edgecolor=None,
        rasterized=False,
    )
    if plot_fit:
        ax.plot(np.log(x), gaussian_arr, color=fit_linecolor, linewidth=linewidth)
    plt.xticks(
        np.log(depth_list),
        (np.array(depth_list) * 100).astype("int"),
        fontsize=fontsize_dict["tick"],
        rotation=45,
    )
    plt.yticks(fontsize=fontsize_dict["tick"])
    ax.set_xlabel("Virtual depth (cm)", fontsize=fontsize_dict["label"])
    ax.set_ylabel("\u0394F/F", fontsize=fontsize_dict["label"])

    plotting_utils.despine()
    

def plot_PSTH(
    fig,
    neurons_df,
    trials_df,
    roi,
    is_closed_loop,
    max_distance=6,
    nbins=20,
    frame_rate=15,
    plot_x=0,
    plot_y=0,
    plot_width=1,
    plot_height=1,
    fontsize_dict={"title": 15, "label": 10, "tick": 10},
    linewidth=3,
):
    """PSTH of a neuron for each depth and blank period.

    Args:
        neurons_df (pd.DataFrame): dataframe with analyzed info of all rois.
        trials_df (pd.DataFrame): dataframe with info of all trials.
        roi (int): ROI number
        is_closed_loop (bool): plotting the closed loop or open loop results.
        max_distance (int, optional): max distance for each trial in meters. Defaults to 6.
        nbins (int, optional): number of bins to bin the activity. Defaults to 20.
        frame_rate (int, optional): imaging frame rate. Defaults to 15.
    """

    # confidence interval z calculation
    ci_range = 0.95
    z = scipy.stats.norm.ppf(1 - ((1 - ci_range) / 2))

    # choose the trials with closed or open loop to visualize
    trials_df = trials_df[trials_df.closed_loop == is_closed_loop]

    depth_list = find_depth_neurons.find_depth_list(trials_df)
    grouped_trials = trials_df.groupby(by="depth")
    trial_number = len(trials_df) // len(depth_list)

    # bin dff according to distance travelled for each trial
    all_means = np.zeros(((len(depth_list) + 1), nbins))
    all_ci = np.zeros(((len(depth_list) + 1), nbins))
    for idepth, depth in enumerate(depth_list):
        all_dff = []
        all_distance = []
        for itrial in np.arange(trial_number):
            dff = grouped_trials.get_group(depth).dff_stim.values[itrial][:, roi]
            rs_arr = grouped_trials.get_group(depth).RS_stim.values[itrial]
            distance = np.cumsum(rs_arr / frame_rate)
            all_dff.append(dff)
            all_distance.append(distance)

        all_dff = np.array([j for i in all_dff for j in i])
        all_distance = np.array([j for i in all_distance for j in i])
        bins = np.linspace(start=0, stop=max_distance, num=nbins + 1, endpoint=True)
        bin_centers = (bins[1:] + bins[:-1]) / 2

        # calculate speed tuning
        bin_means, _, _ = scipy.stats.binned_statistic(
            x=all_distance,
            values=all_dff,
            statistic="mean",
            bins=nbins,
        )

        bin_stds, _, _ = scipy.stats.binned_statistic(
            x=all_distance,
            values=all_dff,
            statistic="std",
            bins=nbins,
        )

        bin_counts, _, _ = scipy.stats.binned_statistic(
            x=all_distance,
            values=all_dff,
            statistic="count",
            bins=nbins,
        )

        all_means[idepth, :] = bin_means

        ci = z * bin_stds / np.sqrt(bin_counts)
        ci[np.isnan(ci)] = 0
        all_ci[idepth, :] = ci

    # Blank dff
    dff = trials_df.dff_blank.values
    rs = trials_df.RS_blank.values

    all_dff = np.array([j for i in dff for j in i[:, roi]])
    rs_arr = np.array([j for i in rs for j in i])
    all_distance = np.cumsum(rs_arr / frame_rate)

    # calculate speed tuning
    bin_means, _, _ = scipy.stats.binned_statistic(
        x=all_distance,
        values=all_dff,
        statistic="mean",
        bins=nbins,
    )

    bin_stds, _, _ = scipy.stats.binned_statistic(
        x=all_distance,
        values=all_dff,
        statistic="std",
        bins=nbins,
    )

    bin_counts, _, _ = scipy.stats.binned_statistic(
        x=all_distance,
        values=all_dff,
        statistic="count",
        bins=nbins,
    )

    all_means[-1, :] = bin_means
    ci = z * bin_stds / np.sqrt(bin_counts)
    ci[np.isnan(ci)] = 0
    all_ci[-1, :] = ci

    ax = fig.add_axes([plot_x, plot_y, plot_width, plot_height]) 
    for idepth, depth in enumerate(depth_list):
        linecolor = basic_vis_plots.get_depth_color(depth, depth_list, cmap=cm.cool.reversed())
        ax.plot(
            bin_centers,
            all_means[idepth, :],
            color=linecolor,
            label=f"{int(depth_list[idepth] * 100)} cm",
            linewidth=linewidth,
        )

        ax.fill_between(
            bin_centers,
            y1=all_means[idepth, :] - all_ci[idepth, :],
            y2=all_means[idepth, :] + all_ci[idepth, :],
            color=linecolor,
            alpha=0.3,
            edgecolor=None,
            rasterized=False,
        )

    ax.plot(
        bin_centers,
        all_means[-1, :],
        color="gray",
        label=f"{int(depth_list[idepth] * 100)} cm",
        linewidth=linewidth,
    )
    ax.fill_between(
        bin_centers,
        y1=all_means[-1, :] - all_ci[-1, :],
        y2=all_means[-1, :] + all_ci[-1, :],
        color="gray",
        alpha=0.3,
        edgecolor=None,
        rasterized=False,
    )

    ax.set_xlabel("Virtual distance (m)", fontsize=fontsize_dict["label"])
    ax.set_ylabel("\u0394F/F", fontsize=fontsize_dict["label"])
    plt.xticks(
        # np.linspace(0, nbins, 3),
        # (np.linspace(0, max_distance, 3) * 100).astype("int"),
        fontsize=fontsize_dict["tick"],
        rotation=45,
    )
    plt.yticks(fontsize=fontsize_dict["tick"])
    plotting_utils.despine()

In [ ]:
VERSION = 1
SAVE_ROOT = '/camp/lab/znamenskiyp/home/shared/presentations/v1_manuscript_2023/ver'+str(VERSION)+'/fig1/'
os.makedirs(SAVE_ROOT, exist_ok=True)

In [ ]:
fig = plt.figure()

# Fig.1A: Motion parallax schema & equation, VR schema
# Fig.1B: Trial structure schema, Spheres with different sizes schema

# Fig.1C: Raster plot of an example neuron
project='hey2_3d-vision_foodres_20220101'
session_name='PZAH8.2h_S20230116'
flexilims_session = flz.get_flexilims_session(project)

vs_df_all, trials_df_all = spheres.sync_all_recordings(
    session_name=session_name,
    flexilims_session=flexilims_session,
    project=project,
    filter_datasets={'anatomical_only':3},
    recording_type="two_photon",
    protocol_base="SpheresPermTubeReward",
    photodiode_protocol=5,
    return_volumes=True,
)

neurons_ds = pipeline_utils.create_neurons_ds(
    session_name=session_name, flexilims_session=flexilims_session, project=None, conflicts="skip"
)
neurons_df = pd.read_pickle(neurons_ds.path_full)

plot_raster_all_depths(
    fig=fig,
    neurons_df=neurons_df,
    trials_df=trials_df_all,
    roi=250,
    is_closed_loop=True,
    max_distance=6,
    nbins=60,
    frame_rate=15,
    vmax=3,
    plot=True,
    plot_x=0,
    plot_y=1,
    plot_width=1,
    plot_height=0.2,
    cbar_width=0.03,
    fontsize_dict={'title':15, 'label':10, 'tick':10},
    );

# Fig.1D: Example PSTH of a single neuron
plot_PSTH(fig=fig,
          neurons_df=neurons_df,
            trials_df=trials_df_all,
            roi=250,
            is_closed_loop=True,
            max_distance=6,
            nbins=20,
            frame_rate=15,
            plot_x=0,
            plot_y=0.67,
            plot_width=0.2,
            plot_height=0.2,
            fontsize_dict={'title':15, 'label':10, 'tick':10},
            linewidth=3,
            )

# Fig.1E: Example depth tuning curve & PSTH of a single neuron
plot_depth_tuning_curve(
    fig=fig,
    neurons_df=neurons_df,
    trials_df=trials_df_all,
    roi=250,
    rs_thr=0.2,
    plot_fit=False,
    linewidth=3,
    linecolor="k",
    fit_linecolor="r",
    closed_loop=1,
    plot_x=0.3,
    plot_y=0.67,
    plot_width=0.2,
    plot_height=0.2,
    fontsize_dict={'title':15, 'label':10, 'tick':10},
    )

# Fig.1F: Example depth tuning curve & PSTH of 3 other neurons
example_rois = [249, 54, 88]
for i, roi in enumerate(example_rois):
    plot_depth_tuning_curve(
        fig=fig,
        neurons_df=neurons_df,
        trials_df=trials_df_all,
        roi=roi,
        rs_thr=0.2,
        plot_fit=False,
        linewidth=3,
        linecolor="k",
        fit_linecolor="r",
        closed_loop=1,
        plot_x=0.3+0.25*(i+1),
        plot_y=0.67,
        plot_width=0.2,
        plot_height=0.2,
        fontsize_dict={'title':15, 'label':10, 'tick':10},
        )
    plt.ylabel('')
    
# Fig.1G: Histogram of preferred depths of all depth-tuned neurons; Raster plot of preferred depths of all neurons (sorted by preferred depth)

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import matplotlib.image as mpimg

# Load the image
img = mpimg.imread('plot_image.png')

# Create a figure
fig, ax = plt.subplots()

# Display the image
ax.imshow(img)
ax.axis('off')  # Turn off axis

plt.show()